In [1]:
import sys
import sqlite3
import numpy as np


IS_PYTHON3 = sys.version_info[0] >= 3
MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB,
    qvec BLOB,
    tvec BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)


    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=False, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.full(4, np.NaN), prior_t=np.full(3, np.NaN), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches,
                              F=np.eye(3), E=np.eye(3), H=np.eye(3),
                              qvec=np.array([1.0, 0.0, 0.0, 0.0]),
                              tvec=np.zeros(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)

        if image_id1 > image_id2:
            matches = matches[:,::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        qvec = np.asarray(qvec, dtype=np.float64)
        tvec = np.asarray(tvec, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H),
             array_to_blob(qvec), array_to_blob(tvec)))

In [2]:
import os
import argparse
import pandas as pd
from sympy import nsolve, Poly, Eq, Matrix, Symbol, solve
from sympy import symbols, symarray

# Open the database.
db = COLMAPDatabase.connect("./only_extrinsics_database.db")

In [3]:
c = db.cursor()
    
table_list = [a for a in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)

[('cameras',), ('sqlite_sequence',), ('images',), ('keypoints',), ('descriptors',), ('matches',), ('two_view_geometries',)]


In [4]:
geometry = [a for a in c.execute("SELECT * FROM two_view_geometries")]
cameras = [cam for cam in c.execute("SELECT * FROM cameras")]
cams = len(cameras)

In [5]:
def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % 2147483647
    image_id1 = (pair_id - image_id2) / 2147483647
    return image_id1, image_id2

In [6]:
tuples = []
matches = []
for pair in geometry:
    pair_id = pair[0]
    match = pair[1]
    im_id1, im_id2 = pair_id_to_image_ids(pair_id)
    F_blob = pair[5]
    if F_blob is not None:
        F_pair = blob_to_array(pair[5], dtype=np.float64)
        tuples.append((int(im_id1), int(im_id2), F_pair))
        matches.append((int(im_id1), int(im_id2), match, F_pair))
    

In [11]:
matchdict = {}
for tup in matches:
    firstcam = tup[0]
    secondcam = tup[1]
    match = tup[2]
    F_pair = tup[3]
    if firstcam not in matchdict:
        matchdict[firstcam] = [(secondcam, match, F_pair)]
    else:
        matchdict[firstcam].append((secondcam, match, F_pair))

In [29]:
# select a few cameras from the list to create equations
# 1, 6, 9, 25, 19, 30
cam_tuples = []
cams_selected = [1, 6, 9]
for i in cams_selected:
    for k in matchdict[i]:
        if k[0] in cams_selected:
            cam_tuples.append((i, k[0], k[2]))

In [39]:
# GOAL - loop through all the pairs to create equations and simulateneously do SVD within the loop

# Task 1 - create all variables first 
intrinsic_vars_mat = {}
intrinsic_vars_list = []
intrinsic_init_list = []

for cam in cams_selected:
    cam_name = "f_" + str(cam)
    v = symarray(cam_name, 2)
    #cam_mat = np.array([[v[0], 0, v[1]],[0, v[2], v[3]], [0, 0, 1]])    
    cam_mat = np.array([[v[0], 0, 0], [0, v[1], 0], [0, 0, 1]])
    #cam_init_list = [5000, 1334//2, 5000, 2048//2]
    cam_init_list = [5000, 5000]
    intrinsic_vars_mat[cam] = cam_mat
    intrinsic_vars_list.extend(list(v))
    intrinsic_init_list.extend(cam_init_list)

In [40]:
print(intrinsic_vars_list)

[f_1_0, f_1_1, f_6_0, f_6_1, f_9_0, f_9_1]


In [47]:
# Task 2 - loop through all the available pairs and concatenate the equations 

def get_initial_estimates(camer_pairs, variables, variables_list, variables_init):
    
    # m*(m-1) = 561 equations (but a few are None) so only 195 
    equations = []
    for pair in camer_pairs:
        im1_id = pair[0]
        im2_id = pair[1]
        
        a1 = variables[im1_id]
        a2 = variables[im2_id]
        #print(k1, k2)
        F_ij_flattened = pair[2]
        #print(F_ij_flattened)
        F_ij = F_ij_flattened.reshape(3,3)
        #print(F_ij)
        u, d, v = np.linalg.svd(F_ij, full_matrices=True)
        #print(d.shape)
        v = v.T

        r = d[0]
        s = d[1]
        u1, u2, u3 = u[:, 0], u[:, 1], u[:, 2]
        v1, v2, v3 = v[:, 0], v[:, 1], v[:, 2]
        
        k1 = a1 @ a1.T
        k2 = a2 @ a2.T
        
        #print(k1, k2)
        
        eq1 = (-1 * r**2 * (v1.T @ (k1 @ v1)) * (u2.T @ (k2 @ u1))) - (r*s* (v1.T @ (k1 @ v2)) * (u2.T @ (k2 @ u2)))
        eq2 = (r**2 * (v1.T @ (k1 @ v1)) *  (u1.T @ (k2 @ u1))) - (s**2 * (v2.T @ (k1 @ v2)) * (u2.T @ (k2 @ u2)))
        
        #print(eq1)
        #print(eq2)
        equations.append(eq1)
        equations.append(eq2)    

    # numerical solver 
    init_solutions = solve(equations, variables_list, dict = True)
    return init_solutions

In [48]:
print(intrinsic_vars_list)
print(intrinsic_init_list)
sols = get_initial_estimates(cam_tuples, intrinsic_vars_mat, intrinsic_vars_list, intrinsic_init_list)

[f_1_0, f_1_1, f_6_0, f_6_1, f_9_0, f_9_1]
[5000, 5000, 5000, 5000, 5000, 5000]


KeyboardInterrupt: 

In [34]:
sols[0]

{f_1_0: 784325.846535251,
 f_1_1: -955933.614342503,
 f_6_0: 756576.309558525,
 f_6_1: 2017278.57331734,
 f_9_0: -2843272.43010356,
 f_9_1: 3270583.89080567}